In [43]:
# importar librerias
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None) # show alll columns

------------------

In [44]:
# get data
df = pd.read_parquet('Cluster_result.parquet')

In [45]:
# info data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59292 entries, 0 to 59291
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   business_name     59292 non-null  object 
 1   rating            59292 non-null  float64
 2   review            59292 non-null  object 
 3   dataset           59292 non-null  object 
 4   review processed  59292 non-null  object 
 5   lenght review     59292 non-null  int64  
 6   lemmatization     59292 non-null  object 
 7   cluster           59292 non-null  int32  
dtypes: float64(1), int32(1), int64(1), object(5)
memory usage: 3.4+ MB


In [5]:
# show data
df.sample(5)

,business_name,rating,review,dataset,review processed,lenght review,lemmatization,cluster
47817,Roar,5.0,A stunning bottling from the Franscioni family...,WineMagazin,stunning bottle franscioni family show exotic ...,35,"[stunning, bottle, franscioni, family, show, e...",0
22343,Gauthier Vineyard,5.0,Loved Gauthier Vineyard. Enjoyed all of their ...,Google,love gauthier vineyard enjoy wine although cha...,26,"[love, gauthier, vineyard, enjoy, wine, althou...",1
2189,Magnavita Cellars,5.0,It's been a long time since I've been wine tas...,Yelp,long time since wine taste ron magnavita excel...,14,"[long, time, since, wine, taste, ron, magnavit...",1
21323,Cunningham Creek Winery,5.0,Great evening playing trivia and enjoying the ...,Google,great evening play trivia enjoy beautiful vine...,7,"[great, evening, play, trivia, enjoy, beautifu...",1
10736,Tempo Vino Winery of Kalamazoo,4.0,I was at this place couple of weeks ago. Most ...,Google,place couple week ago interesting part make wi...,23,"[place, couple, week, ago, interesting, part, ...",1


In [ ]:
# en este caso la etiqueta 0 en el campo cluster serian conocedores de vino, al ejecutar el notebook GoogleYelp_Cluster este valor puede cambiar a 0, verificar siempre

------------

# Classification

In [46]:
# Seleccionar variables predictoras X - variable a predecir y
X = df['review processed']
y = df['cluster']

In [47]:
# validar balance
df['cluster'].value_counts()

0    35070
1    24222
Name: cluster, dtype: int64

In [48]:
# proporcion etiqueteas positivas variable objetivo
print('Proporcion etiquetas positivas variable objetivo: ', y.sum()/y.count())

Proporcion etiquetas positivas variable objetivo:  0.4085205424003238


In [49]:
# obtener datos de entrenamiento - datos de prueba (split)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1, stratify=y)

------------

### MultinomialNB

In [74]:
# tfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2))

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [75]:
from sklearn.naive_bayes import MultinomialNB
clf_model = MultinomialNB(alpha=0.1)

clf_model.fit(X_train_tfidf, y_train)

MultinomialNB(alpha=0.1)

In [76]:
# evaluar modelo
pred_test = clf_model.predict(X_test_tfidf)

In [56]:
# exactitud, accuracy - total datos
from sklearn.metrics import accuracy_score

print('Exactitud:', accuracy_score(y_test, pred_test))

Exactitud: 0.988363268403744


In [57]:
# precision
from sklearn.metrics import precision_score

print('Presicion:', precision_score(y_test, pred_test))

Presicion: 0.9782564519406625


In [58]:
# exhaustividad, sensibilidad, recall - total datos
from sklearn.metrics import recall_score

print('Exhaustividad:', recall_score(y_test, pred_test))

Exhaustividad: 0.9936016511867906


In [59]:
# reporte de clasificacion
from sklearn.metrics import classification_report

reporte = classification_report(y_test, pred_test, target_names=['pro', 'non_pro'])
print('reporte de clasificacion:')
print(reporte)

reporte de clasificacion:
              precision    recall  f1-score   support

         pro       1.00      0.98      0.99      7014
     non_pro       0.98      0.99      0.99      4845

    accuracy                           0.99     11859
   macro avg       0.99      0.99      0.99     11859
weighted avg       0.99      0.99      0.99     11859



-----------

### test con datos de prueba del dataset

In [60]:
len(X_test)

11859

In [61]:
index = X_test.index[764]
index

54279

In [62]:
df.loc[index]

business_name                                               Treleaven
rating                                                            4.0
review              Violets mingle with notes of dark char and alm...
dataset                                                   WineMagazin
review processed    violet mingle note dark char almond perfume nu...
lenght review                                                      28
lemmatization       [violet, mingle, note, dark, char, almond, per...
cluster                                                             0
Name: 54279, dtype: object

In [63]:
review = df['review processed'][index]
review

'violet mingle note dark char almond perfume nuanced cabernet franc palate delicate sour cherry flavor flavor subdue swathe earth spice brisk acidity finish mouthful penetrate finely textured tannin'

In [64]:
# test
clf_model.predict([review])

array([0])

In [65]:
print(clf_model.predict([review]))

[0]


In [69]:
import pickle

with open('model.pickle', 'wb') as f:
    pickle.dump(clf_model, f)

In [67]:
from joblib import dump

dump(clf_model, 'classification_MultinomialNB') 

['classification_MultinomialNB']

In [70]:
from joblib import load

clf_model = load('classification_MultinomialNB')

text = 'violet mingle note dark char almond perfume nuanced cabernet franc palate delicate sour cherry flavor flavor subdue swathe earth spice brisk acidity finish mouthful penetrate finely textured tannin'

clf_model.predict([text])

array([0])